# Walking through a multilayer perceptron's computation

## Code to accompany *Deep learning methods for mobile sensing*

#### Please contact Christopher J. Urban at cjurban@live.unc.edu with any questions

In this notebook, we'll walk through how a multilayer perceptron (MLP; a kind of artificial neural network) processes input data.

The notebook is written in Python, so basic familiarity with Python could make things easier to follow, although it's not necessary.

First, we'll import some necessary packages. "sklearn" refers to __[scikit-learn](https://scikit-learn.org/stable/)__, which is a very useful Python package for many machine learning applications.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import neural_network
from sklearn.metrics import accuracy_score

Next, we'll simulate some data according to the following equation (also described in the text):
\begin{align}
    \begin{split}
        \text{NUD} &\sim \text{Bernoulli}(0.5) \\
        \text{AUD} &\sim \text{Bernoulli}(0.4) \\
        \text{MDD} &\sim \text{Bernoulli}\Big(f\big(-2 + 4 \cdot  I(\text{NUD} = \text{AUD})\big)\Big),
    \end{split}
\end{align}
where $I(\cdot)$ is the indicator function, NUD is nicotine use disorder, AUD is alcohol use disorder, and MDD is major depressive disorder. In words, folks in this population have a $50\%$ and $40\%$ chance of having NUD and AUD, respectively, while chances of having MDD are high if you have both or neither NUD and AUD but low if you have only one of NUD or AUD. In other words, having MDD is nonlinearly related to having NUD and AUD.

In [2]:
np.random.seed(123)
N = 5000
nud = np.random.binomial(1, 0.5, [N])
aud = np.random.binomial(1, 0.4, [N])
mdd = np.random.binomial(1, 1/(1 + np.exp(2-4 * (aud == nud))), [N])

After simulating the data, we'll split it into a training and test set, then fit our MLP on the training set and compute its accuracy on the test set. Notice that we set several random seeds to ensure that results are reproducible. 

Note: There is also a convergence warning, which tells us that the scikit-learn default convergence criterion (i.e., criterion that determines when to stop the iterative fitting procedure) wasn't met. Our MLP already has good accuracy, so this isn't really a problem.

In [3]:
X = np.concatenate([np.expand_dims(aud, axis = 1), np.expand_dims(nud, axis = 1)], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, mdd, test_size = 0.25, random_state = 234)

# Fit MLP
mlp_clf = neural_network.MLPClassifier(hidden_layer_sizes = (4, 4),
                                      random_state = 456)
mlp_pred = mlp_clf.fit(X_train, y_train).predict(X_test)
mlp_acc = accuracy_score(y_test, mlp_pred)
print("MLP accuracy = ", mlp_acc)

MLP accuracy =  0.8696


/Users/gateslab/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


We'll also fit a logistic regression classifier to compare its performance to the MLP's. Its accuracy is much lower because, unlike the MLP, it fails to model the nonlinear relationship between the predictors and the outcomes.

In [4]:
# Fit logistic regression
lr_clf = linear_model.LogisticRegression(penalty = "none",
                                         solver = "newton-cg")
lr_pred = lr_clf.fit(X_train, y_train).predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print("Logistic regression accuracy = ", lr_acc)

Logistic regression accuracy =  0.5664


Finally, we step through how the fitted MLP processes input data (see text for more details). Here, we extract the fitted regression weights and intercepts, then use them to manually compute the predicted probability of having MDD for a specific individual. All fitted parameters and intermediate outputs are printed out below. Fitted values may be slightly different from those reported in the text.

In [5]:
# First layer computations
W1 = mlp_clf.coefs_[0]
b1 = mlp_clf.intercepts_[0]
x = np.array([1, 0]) 
a1 = np.matmul(x, W1) + b1
h1 = np.maximum(a1, 0) # ReLU activation

# Second layer computations
W2 = mlp_clf.coefs_[1]
b2 = mlp_clf.intercepts_[1]
a2 = np.matmul(h1, W2) + b2
h2 = np.maximum(a2, 0) # ReLU activation

# Output computations
w3 = mlp_clf.coefs_[2]
b3 = mlp_clf.intercepts_[2]
a3 = np.matmul(h2, w3) + b3
y_hat = 1 / (1 + np.exp(-a3)) # Sigmoid final activation

print("First layer computations")
print("x  =", np.array2string(x.transpose(), prefix = "x  ="))
print("W1 =", np.array2string(np.around(W1.transpose(), 2), prefix = "W1 ="))
print("b1 =", np.array2string(np.around(np.expand_dims(b1.transpose(), axis = 1), 2), prefix = "b1 ="))
print("a1 =", np.array2string(np.around(np.expand_dims(a1.transpose(), axis = 1), 2), prefix = "a1 ="))
print("h1 =", np.array2string(np.around(np.expand_dims(h1.transpose(), axis = 1), 2), prefix = "h1 ="))

print("\nSecond layer computations")
print("W2 =", np.array2string(np.around(W2.transpose(), 2), prefix = "W2 ="))
print("b2 =", np.array2string(np.around(np.expand_dims(b2.transpose(), axis = 1), 2), prefix = "b2 ="))
print("a2 =", np.array2string(np.around(np.expand_dims(a2.transpose(), axis = 1), 2), prefix = "a2 ="))
print("h2 =", np.array2string(np.around(np.expand_dims(h1.transpose(), axis = 1), 2), prefix = "h2 ="))

print("\nOutput computations")
print("w3    =", np.array2string(np.around(w3, 2), prefix = "w3    ="))
print("b3    =", np.array2string(np.around(b3, 2), prefix = "b3    ="))
print("a3    =", np.array2string(np.around(a3, 2), prefix = "a3    ="))
print("y_hat =", np.array2string(np.around(y_hat, 2), prefix = "y_hat ="))

First layer computations
x  = [1 0]
W1 = [[-0.   -0.  ]
     [-0.    0.  ]
     [ 1.25  1.25]
     [ 1.25  1.27]]
b1 = [[-0.64]
     [-0.7 ]
     [-1.26]
     [ 0.  ]]
a1 = [[-0.64]
     [-0.7 ]
     [-0.  ]
     [ 1.25]]
h1 = [[0.  ]
     [0.  ]
     [0.  ]
     [1.25]]

Second layer computations
W2 = [[ 0.    0.   -0.    0.  ]
     [-0.   -0.   -0.   -0.  ]
     [ 0.    0.   -0.    0.  ]
     [-0.    0.   -2.67  1.32]]
b2 = [[-0.52]
     [-0.63]
     [-0.5 ]
     [-0.  ]]
a2 = [[-0.52]
     [-0.63]
     [-0.5 ]
     [ 1.65]]
h2 = [[0.  ]
     [0.  ]
     [0.  ]
     [1.25]]

Output computations
w3    = [[ 0.  ]
        [ 0.  ]
        [-0.  ]
        [-2.32]]
b3    = [1.77]
a3    = [-2.06]
y_hat = [0.11]
